# Example workflow calculation. Data reprocessing CPU

In this example we show the calculation to estimate the CPU needed to process data on the Grid at Tier-1s and Tier-2s. 

In this configuration, we process the data collected each year twice, and the data from the previous years in the same Run once. 

## Input parameters
CPU is measured in seconds/event in a CPU core  
Sizes (for Disk and Tape) in kBytes/event 
Atomic unit is the event. 

For each year (between 2015 and 2029) the computing model parameters can vary. Computing model parameters are stored in the *inP* dictionary:
   - RealRecoCPU is the # of seconds of CPU to run reconstruction for one event
   - CPUEffReco is the CPU efficiency achieved
   - SameYrReprData is how many times data from the current year is processed
   - PrevYrReprData is how many times data from previous years is processed
   - ReprYrs is a list of years for which data is reprocessed in the current year (eg in 2018 we may wish to process 2015, 2016, 2017)

The operation parameters from the LHC and the ATLAS trigger are stored in the *LhcP* dictionary:
   - Rate is the averge rate of output from the High Level Trigger (HLT)
   - RunTime is how many seconds we expect the LHC to operate in a given year
   
Parameters are realistic, but somewhat invented. They reflect the growth in CPU/event needed to process events at higher lumi and higher <mu>

In [1]:
#
# inP is the global variable to store all the performance parameters.
# LhcP is the global variable to store the LHC running parameters
#
inP={}
LhcP={}

years=range(2015,2029)
# Constants
Seconds_in_year=365*86400



In [2]:
#
# Parameters to be used for this examples. 
#

def costModelPars():
    #
    # LHC operating parameters
    # Rate in Hz and RunTime in sec
    #
    LhcP[2015]={'Rate':1000, 'RunTime':3.0E06} # Run 2
    LhcP[2016]={'Rate':1000, 'RunTime':7.3E06} # Run 2
    LhcP[2017]={'Rate':1000, 'RunTime':7.3E06} # Run 2
    LhcP[2018]={'Rate':1000, 'RunTime':7.3E06} # Run 2
    LhcP[2019]={'Rate':0, 'RunTime':0.0} # LS2
    LhcP[2020]={'Rate':0, 'RunTime':0.0} # LS2
    LhcP[2021]={'Rate':1000, 'RunTime':7.3E06} # Run 3
    LhcP[2022]={'Rate':1000, 'RunTime':7.3E06} # Run 3
    LhcP[2023]={'Rate':1000, 'RunTime':7.3E06} # Run 3
    LhcP[2024]={'Rate':0, 'RunTime':0.0} # LS3
    LhcP[2025]={'Rate':0, 'RunTime':0.0} # LS3
    LhcP[2026]={'Rate':10000, 'RunTime':7.3E06} # Run 4
    LhcP[2027]={'Rate':10000, 'RunTime':7.3E06} # Run 4
    LhcP[2028]={'Rate':10000, 'RunTime':7.3E06} # Run 4
    
    #
    # CPU parameters
    # RealRecoCPU: time to reconstruct event
    # AODsize: size of the reconstructed event (kB)
    # RAWsize: size of the raw dat of a single event (kB)
    inP[2015]={'RealRecoCPU':20, 'AODSize': 250, 'RAWSize': 1000}
    inP[2016]={'RealRecoCPU':20, 'AODSize': 250, 'RAWSize': 1000}
    inP[2017]={'RealRecoCPU':25, 'AODSize': 250, 'RAWSize': 1000}
    inP[2018]={'RealRecoCPU':25, 'AODSize': 250, 'RAWSize': 1000}
    inP[2019]={'RealRecoCPU':25, 'AODSize': 250, 'RAWSize': 1000}
    inP[2020]={'RealRecoCPU':25, 'AODSize': 250, 'RAWSize': 1000}
    inP[2021]={'RealRecoCPU':50, 'AODSize': 350, 'RAWSize': 1000}
    inP[2022]={'RealRecoCPU':50, 'AODSize': 350, 'RAWSize': 1000}
    inP[2023]={'RealRecoCPU':50, 'AODSize': 350, 'RAWSize': 1000}
    inP[2024]={'RealRecoCPU':50, 'AODSize': 350, 'RAWSize': 1000}
    inP[2025]={'RealRecoCPU':130, 'AODSize': 1000, 'RAWSize': 5000}
    inP[2026]={'RealRecoCPU':130, 'AODSize': 1000, 'RAWSize': 5000}
    inP[2027]={'RealRecoCPU':130, 'AODSize': 1000, 'RAWSize': 5000}
    inP[2028]={'RealRecoCPU':130, 'AODSize': 1000, 'RAWSize': 5000}
    
    # Assume CPU efficiency is at 75% 
    for year in years:
        inP[year].update({'CPUEffReco':0.75})
        
    #
    # Reprocessing parameters
    #
    for year in years:
        inP[year].update({'SameYrReprData':2})
        inP[year].update({'PrevYrReprData':1})

    inP[2015].update({'ReprYrs':[]})
    inP[2016].update({'ReprYrs':[2015]})
    inP[2017].update({'ReprYrs':[2015,2016]})
    inP[2018].update({'ReprYrs':[2015, 2016, 2017]})
    inP[2019].update({'ReprYrs':[2015, 2016, 2017, 2018]})
    inP[2020].update({'ReprYrs':[]})
    inP[2021].update({'ReprYrs':[]})
    inP[2022].update({'ReprYrs':[2021]})
    inP[2023].update({'ReprYrs':[2021,2022]})
    inP[2024].update({'ReprYrs':[2021,2022,2023]})
    inP[2025].update({'ReprYrs':[]})
    inP[2026].update({'ReprYrs':[]})
    inP[2027].update({'ReprYrs':[2026]})
    inP[2028].update({'ReprYrs':[2026,2027]})  
    

    

## Calculations for data reprocessing
For each year we calculate the CPU needed to reprocess data. This is the sum of the CPU needed to reprocess the current year data (twice) and the CPU needed to reprocess some of the previous years data. 

The assumption for the reprocessing parameters are in the inP dictionary. The LHC parameters are used to calculate the number of events that are collected each year. (So this will be zero during long shutdown years)

In [3]:
# Output quantities
NDataEvts={}
CPUOneRound={}
CPURepr={}

# Use default values for parameters
costModelPars()

for year in years:
    print (year, inP[year])

2015 {'RealRecoCPU': 20, 'AODSize': 250, 'RAWSize': 1000, 'CPUEffReco': 0.75, 'SameYrReprData': 2, 'PrevYrReprData': 1, 'ReprYrs': []}
2016 {'RealRecoCPU': 20, 'AODSize': 250, 'RAWSize': 1000, 'CPUEffReco': 0.75, 'SameYrReprData': 2, 'PrevYrReprData': 1, 'ReprYrs': [2015]}
2017 {'RealRecoCPU': 25, 'AODSize': 250, 'RAWSize': 1000, 'CPUEffReco': 0.75, 'SameYrReprData': 2, 'PrevYrReprData': 1, 'ReprYrs': [2015, 2016]}
2018 {'RealRecoCPU': 25, 'AODSize': 250, 'RAWSize': 1000, 'CPUEffReco': 0.75, 'SameYrReprData': 2, 'PrevYrReprData': 1, 'ReprYrs': [2015, 2016, 2017]}
2019 {'RealRecoCPU': 25, 'AODSize': 250, 'RAWSize': 1000, 'CPUEffReco': 0.75, 'SameYrReprData': 2, 'PrevYrReprData': 1, 'ReprYrs': [2015, 2016, 2017, 2018]}
2020 {'RealRecoCPU': 25, 'AODSize': 250, 'RAWSize': 1000, 'CPUEffReco': 0.75, 'SameYrReprData': 2, 'PrevYrReprData': 1, 'ReprYrs': []}
2021 {'RealRecoCPU': 50, 'AODSize': 350, 'RAWSize': 1000, 'CPUEffReco': 0.75, 'SameYrReprData': 2, 'PrevYrReprData': 1, 'ReprYrs': []}
202

In [4]:

print ("year \t NEvnt \t CPU1 \t CPU2 \t CPUprv \t CPUall")
print ("\t *10^9 \t kcores kcores \t kcores \t kcores")

for year in years:
# Total number of data events
    NDataEvts[year]=LhcP[year]['Rate']*LhcP[year]['RunTime'] # Number of data events recorded

# Number of CPU cores needed to process all the data of a given year once
    CPUOneRound[year]=NDataEvts[year]*inP[year]['RealRecoCPU']/inP[year]['CPUEffReco']/Seconds_in_year  

# Number of CPU cores needed to process all the data of a given year 
    CPUCurrYr=CPUOneRound[year]*inP[year]['SameYrReprData']

# Calculate the number of CPU cores needed to reprocess previous years (this is more complex)
    CPUPrevYr=0.
    for yr in inP[year]['ReprYrs']:
        CPUPrevYr += CPUOneRound[yr]*inP[year]['PrevYrReprData']

# Total CPU needed for reprocessing
    CPURepr[year]=CPUCurrYr+CPUPrevYr

    print(str(year) + '\t %5.1f'%(NDataEvts[year]/1E9) +
                      '\t %5.0f'%(CPUOneRound[year]/1000.) + 
                      '\t %5.0f'%(CPUCurrYr/1000.) + 
                      '\t %5.0f'%(CPUPrevYr/1000.) +
                      '\t \t %5.0f'%(CPURepr[year]/1000.) )


year 	 NEvnt 	 CPU1 	 CPU2 	 CPUprv 	 CPUall
	 *10^9 	 kcores kcores 	 kcores 	 kcores
2015	   3.0	     3	     5	     0	 	     5
2016	   7.3	     6	    12	     3	 	    15
2017	   7.3	     8	    15	     9	 	    24
2018	   7.3	     8	    15	    16	 	    32
2019	   0.0	     0	     0	    24	 	    24
2020	   0.0	     0	     0	     0	 	     0
2021	   7.3	    15	    31	     0	 	    31
2022	   7.3	    15	    31	    15	 	    46
2023	   7.3	    15	    31	    31	 	    62
2024	   0.0	     0	     0	    46	 	    46
2025	   0.0	     0	     0	     0	 	     0
2026	  73.0	   401	   802	     0	 	   802
2027	  73.0	   401	   802	   401	 	  1204
2028	  73.0	   401	   802	   802	 	  1605


In [5]:
import numpy as np
import plotly 
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

#print [CPURepr[x] for x in years] 

data1 = go.Scatter(x=np.array(years), y=[CPURepr[x] for x in years], mode='lines+markers', name='CPU')
layout = go.Layout(title="CPU needs (#cores) per year")
data = [data1,]
plotly.offline.iplot({"data": data, "layout": layout})

# Exercise 1

Try now to calculate how much CPU is needed for producing simulated data (Monte Carlo). 
Monte Carlo production happens in 3 steps: 

1) Event Generation: from a random seed one produces a "perfect event" (no interaction with the detector). This requires in average 10 seconds of CPU per event. 

2) Detector Simulation: you simulate the interactions of the particles with the instrument (the detector). This might be very time consuming. Consider 350 seconds of CPU/event 

3) Event Reconstruction: similar to reconstruction for real data, use the same number of seconds/event 

Every year you need to simulate the same # of events as the real data you collect. However, you need to reconstruct them twice every year, to adjust for the correct data taking conditions

In the years when you have no data taking, you still need to produce Monte Carlo samples. You need to double up the event statistic you produced in the previous years of Run. For example if during LHC Run-2 (2015, 2016, 2017) you produced N events, in the 2019,2020 shutdown you need to produce other N.  

Do you need more CPUs for data or Monte Carlo? 


In [6]:
# Update with number of sim events to generate and reconstruct per year
LhcP[2015].update({'NSimEv' : LhcP[2015]['Rate']*LhcP[2015]['RunTime']})
LhcP[2016].update({'NSimEv' : LhcP[2016]['Rate']*LhcP[2016]['RunTime']})
LhcP[2017].update({'NSimEv' : LhcP[2017]['Rate']*LhcP[2017]['RunTime']})
LhcP[2018].update({'NSimEv' : LhcP[2018]['Rate']*LhcP[2018]['RunTime']})

LhcP[2019].update({'NSimEv' : (LhcP[2015]['NSimEv'] + 
                               LhcP[2016]['NSimEv'] +
                               LhcP[2017]['NSimEv'] + 
                               LhcP[2018]['NSimEv'] ) /2 })
LhcP[2020].update({'NSimEv' : LhcP[2019]['NSimEv']})

LhcP[2021].update({'NSimEv' : LhcP[2021]['Rate']*LhcP[2021]['RunTime']})
LhcP[2022].update({'NSimEv' : LhcP[2022]['Rate']*LhcP[2022]['RunTime']})
LhcP[2023].update({'NSimEv' : LhcP[2023]['Rate']*LhcP[2023]['RunTime']})

LhcP[2024].update({'NSimEv' : (LhcP[2021]['NSimEv'] + 
                               LhcP[2022]['NSimEv'] + 
                               LhcP[2023]['NSimEv'] ) /2 })
LhcP[2025].update({'NSimEv' : LhcP[2024]['NSimEv']})

LhcP[2026].update({'NSimEv' : LhcP[2026]['Rate']*LhcP[2026]['RunTime']})
LhcP[2027].update({'NSimEv' : LhcP[2027]['Rate']*LhcP[2027]['RunTime']})
LhcP[2028].update({'NSimEv' : LhcP[2028]['Rate']*LhcP[2028]['RunTime']})

In [16]:
evgen_time = 10
evsim_time = 350
NSimEvts = {}
CPUReprSim = {}

for year in years:
    
    # Total number of data events to simulate
    NSimEvts[year] = LhcP[year]['NSimEv']
    
    # Define  "Event Reconstruction time for sim events"
    inP[year].update({'SimEvRecoCPU':inP[year]['RealRecoCPU']})

    #Total theoretical time for generating and analyzing montecarlo data
    theor_comput_time = NSimEvts[year] * (evgen_time + evsim_time + 2*inP[year]['SimEvRecoCPU'])
        
    # Number of CPU cores needed to process all the data of a given year once
    CPUOneRound[year] = theor_comput_time / (inP[year]['CPUEffReco']*Seconds_in_year) 

    # Number of CPU cores needed to process all the data of a given year 
    CPUReprSim[year] = CPUOneRound[year] * inP[year]['SameYrReprData']
    
    
#plot CPUs needed to 
plotly.offline.init_notebook_mode(connected=True)

print ([CPUReprSim[x] for x in years]) 

data1 = go.Scatter(x=np.array(years), y=[CPUReprSim[x] for x in years], mode='lines+markers', name='CPU')
layout = go.Layout(title="CPU needs (#cores) per year")
data = [data1,]
plotly.offline.iplot({"data": data, "layout": layout})

[101471.33434804667, 246913.58024691357, 253086.41975308643, 253086.41975308643, 431633.6884830035, 431633.6884830035, 283950.6172839506, 283950.6172839506, 283950.6172839506, 425925.9259259259, 574074.074074074, 3827160.4938271604, 3827160.4938271604, 3827160.4938271604]


## Exercise 2

Try now to calculate how much disk and tape space you need in order to store the data. 

1) You need to store 2 copies of the RAW data on tape. You need to store 10% of the RAW data of the current year on disk 

2) For DATA, you need to store one copy of the AODs on tape. You need to store 2 copies of the AODs on disk for the AODs of the current year. You need to store one copy of the AODs of the previous years on disk. 

3) For Monte Carlo, you need to store the ouput of Detector Simulation (HITS) on tape and keep 10% of it on disk. One event of HITS is approximately 1MB. You also need to keep 1 copy of the AODs on disk. One simulated AOD is 30% larger in size than a data AOD.  

4) In order to facilitate analysis, you need to consider 30% attitional space for AODs to be available in the form of a cache, to facilitate data access

5) You can never fill up disk at 100%. Leave a 10% safety margin. 





In [9]:
mem_AOD_prev_years = 0
HITS_size = 1000 #kB

true_tape_mem = 0
mc_tape_mem = 0
true_disk_mem = 0
mc_disk_mem = 0
tot_tape_mem = {}
tot_disk_mem = {}

for year in years:

    # Compute Tape memory
    true_tape_mem = NDataEvts[year] * (2*inP[year]['RAWSize'] + inP[year]['AODSize']*1.3)
    mc_tape_mem = NSimEvts[year] * HITS_size
    tot_tape_mem[year] = (true_tape_mem + mc_tape_mem) * 1.1 / 1000000000 #conversion in TB
    
    # Compute Disk memory
    true_disk_mem = NDataEvts[year] * (0.1*inP[year]['RAWSize'] + 2*inP[year]['AODSize']*1.3) + mem_AOD_prev_years
    mc_disk_mem = NSimEvts[year] * (0.1*HITS_size + inP[year]['AODSize']*1.3*1.3)
    tot_disk_mem[year] = (true_disk_mem + mc_disk_mem) * 1.1 / 1000000000  #conversion in TB
    
    # update size of previous years events to save on disk
    mem_AOD_prev_years += NDataEvts[year] * inP[year]['AODSize']*1.3


{2015: 4199.250000000001, 2016: 11290.675, 2017: 13900.425000000001, 2018: 16510.175000000003, 2019: 16057.387500000003, 2020: 16057.387500000003, 2021: 22564.795, 2022: 26218.445000000003, 2023: 29872.095000000005, 2024: 28191.817500000005, 2025: 41423.25, 2026: 412529.70000000007, 2027: 516919.70000000007, 2028: 621309.7}


In [17]:
plotly.offline.init_notebook_mode(connected=True) 

print ([tot_tape_mem[x] for x in years]) 

data1 = go.Scatter(x=np.array(years), y=[tot_tape_mem[x] for x in tot_tape_mem], mode='lines+markers', name='Tape')
layout = go.Layout(title="Tape needs (in TB) per year")
data = [data1,]
plotly.offline.iplot({"data": data, "layout": layout})

[10972.5, 26699.750000000004, 26699.750000000004, 26699.750000000004, 13695.000000000002, 13695.000000000002, 27743.650000000005, 27743.650000000005, 27743.650000000005, 12045.0, 12045.0, 987690.0000000001, 987690.0000000001, 987690.0000000001]


In [18]:
plotly.offline.init_notebook_mode(connected=True)

print ([tot_disk_mem[x] for x in years]) 

data1 = go.Scatter(x=np.array(years), y=[tot_disk_mem[x] for x in years], mode='lines+markers', name='Disk')
layout = go.Layout(title="Disk needs (in TB) per year")
data = [data1,]
plotly.offline.iplot({"data": data, "layout": layout})

[4199.250000000001, 11290.675, 13900.425000000001, 16510.175000000003, 16057.387500000003, 16057.387500000003, 22564.795, 26218.445000000003, 29872.095000000005, 28191.817500000005, 41423.25, 412529.70000000007, 516919.70000000007, 621309.7]


# Exercise 3

One CPU core costs 100 Euros. One TB of disk costs 40 Euros. One TB of tape costs 10 Euros
What is the largest cost according to this simplified Computing Model 

In [20]:
cpu_cost = 100
tape_cost = 10
disk_cost = 40
tot_cost = {}
for year in years:
    tot_cost[year] = (CPURepr[year] + CPUReprSim[year])*cpu_cost + tot_tape_mem[year]*tape_cost + tot_disk_mem[year]*disk_cost

# plot max cost        
plotly.offline.init_notebook_mode(connected=True)

print ([tot_cost[x] for x in years]) 

data1 = go.Scatter(x=np.array(years), y=[tot_cost[x] for x in years], mode='lines+markers', name='Tot Cost')
layout = go.Layout(title="Largest Cost (in euros) per year")
data = [data1,]
plotly.offline.iplot({"data": data, "layout": layout})

[10932185.1065449, 26898228.761796042, 28545828.638339255, 29421823.576610856, 46356786.51133096, 43942614.34830035, 32661509.781481486, 34350865.65802469, 36040221.53456789, 48470344.92222222, 59184787.4074074, 489341050.962963, 533640107.7530864, 577939164.5432099]
